In [22]:
import numpy as np
import pandas as pd
import csv
import altair as alt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from collections import defaultdict
import csv
import ast
from wikipedia2vec import Wikipedia2Vec
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

alt.data_transformers.enable('default', max_rows=None)  #renders if the df has more than 5000 rows

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


DataTransformerRegistry.enable('default')

In [2]:
def def_value(): 
    return 0

entities_freq = defaultdict(def_value)
entities_flag = {}
entities_link_prob = set()
input_file = "E:/Gianluca/Master Big Data Pisa/Progetto_Finale/Agricolo/Facebook/csv_docs_def/comments_entities.csv"
count = 0
with open(input_file, 'r', encoding='utf-8', newline='') as handle_r:
    reader = csv.DictReader(handle_r)
    for row in reader:
        row_entities = ast.literal_eval(row["entities"])
        if len(row_entities) == 0:
            continue
        else:
            for entity in row_entities:
                try:
                    if entity["title"] not in entities_flag:
                        entities_flag[entity["title"]] = 0
                    if entity["link_probability"] == 1 and entity["rho"] == 0.5:
                        entities_link_prob.add(entity["title"])
                    if entity["link_probability"] != 1 or entity["rho"] != 0.5:
                        entities_flag[entity["title"]] = 1
                except:
                    pass
                try:
                    entities_freq[entity["title"]] = entities_freq[entity["title"]] + 1
                except:
                    count = count + 1

In [3]:
ENTITIES_BIN = {"Tutte lo vogliono", "Comitato Olimpico Internazionale", "VI secolo", "Homo sapiens", "Avanti!", "Marco Fanno", "Roberto Magari", "Forza", 
"Hazdayi Penso", "Grazie a tutti", "Worstward Ho", "Con un ma e con un se", "Piacé", 
"Verissimo - Tutti i colori della cronaca", "Tutte e nessuna", "Così è (se vi pare)", "Bene (filosofia)",
"Buona giornata", "Con voi (singolo)", "Buona Domenica", "Uno di noi (serie televisiva)", "Essere", "Sistema internazionale di unità di misura",
"Grazie mille", "Tempo", "Grazia (teologia)", "PHP", "Fata", "Provincia di Carbonia-Iglesias", "Cappello a cono di paglia",
"Ora", "Dalla vostra parte", "Silicio", "Po", "Uno come te", "Parole sante", "Cio (città antica)", "Mica", "L'Ora", "Giovanni Battista Zitti",
"Anno", "La nostra vita (film)", "Chi (periodico)", "Viva l'Italia (film)", "Teymur Məmmədov", "Partito dei Comunisti Italiani",
"Grazie (mitologia)", "Gli orgogliosi", "Eccellenza (calcio)", "W l'Italia", "Sei bellissima/Spiagge di notte", "Era (tempo)", "Non Mollare",
"Giovanni de Primis", "Irene Grandi", "Ugo di La Certa", "La deve smettere!", "Uno di noi (programma televisivo)", "Parola", "Simbolo",
"Svegliatevi!", "E già", "Peccato", "Tè", "Ora o mai più (film)", "Io (psicologia)", "Tipo (Pokémon)", "Selenio", "La Porcherie",  
"Fa (nota)", "Si (nota)", "Mese", "Molla", "Augure", "Pensiero", "Dino Grandi", "Giovanni La Magna", "Berillio",
"Tutti a casa", "Buona vita", "Tutto tace", "Rainer Troppa", "Bla Bla Bla", "Dar'a", "Guido Possa", "Federico Paghi", ".com",
"A testa alta (film 2004)", "Cinema", "Ragazzi (atletica leggera)", "Giovanni Fondelli", "Noi con voi", "Una di troppo", "A casa nostra",
"Io (filosofia)", "Assolutamente sì", "Grazie a tutti (programma televisivo)", "Belisario Vinta", "Poesia", "Buonasera con...",
"Arte", "Ferro", "E un'altra cosa...", "Alfabeto fonetico internazionale", "Mario Abbiate", "Oggi (rivista)", "Sé (coscienza)", "Nulla", "Domenica",
"Marco D'Altrui", "VI secolo a.C.", "Non è per sempre", "Non più di uno", "Rosario Vieni", "Adesso o mai", "Specie", "I migliori del mondo",
"Con il cuore", "Grazie di tutto (film)", "Da stasera/La vita vista da un aereo", "Fondo Ambiente Italiano", "Mettā", "Sai xChé?", "E non sai",
"Titanio", "Sistema consuetudinario statunitense", "World Wide Web", "Categoria (Pokémon)", "Calcio (elemento chimico)", "Basso elettrico",
"Genere letterario", "Lei è (album)", "Aristotele", "Gmail", "La stessa barca", "Mano", "Buon pomeriggio (Canale 5)",
"Per fortuna", "Non-A", "Testa", "Basso (voce)", "Grado giorno", "Loro ed io", "Casa da gioco", "Io ci sarò (Piero Pelù)", "Il meglio di...", 
"Pronome personale", "Questo e quello", "Buona giornata e...", "I più grandi di tutti",
"VI Legislatura della Repubblica Italiana", "Non Abbiamo Bisogno", "Forti e Liberi", "Mente", "Saro (generale)",
"Tastiera (musica)", "Se per te"}

for title in entities_flag :
    if entities_flag[title] == 0:
        ENTITIES_BIN.add(title)

CORRECTIONS = {
    "Signoria cittadina": "Dio", 
    "Nazionale di calcio della Francia": "Francia", 
    "À San Remo": "Sanremo", 
    "Virtual Address eXtension": "Vaccino", 
    "Lingua francese": "Francia", 
    "Canto (musica)": "Manifestazione", 
    "Il Popolo": "Popolo", 
    "Regno di Sardegna": "Sardegna", 
    "Europa (astronomia)": "Europa", 
    "Civiltà romana": "Roma", 
    "Costa": "Prezzo", 
    "Pagliacci (opera)": "Pagliaccio", 
    "Giorgia (cantante 1971)": "Giorgia Meloni", 
    "Massa (fisica)": "Massa (sociologia)", 
    "Nazionale di calcio della Germania": "Germania", 
    "Trattore d'artiglieria": "Trattore agricolo",
    "Nazionale di calcio dell'Italia": "Italia", 
    "Germania nazista": "Germania", 
    "Lingua italiana": "Italiani", 
    "Regno d'Italia": "Italia", 
    "Cantante": "Manifestazione", 
    "Siamo italiani": "Italiani",
    "Che Dio ci aiuti": "Dio", 
    "Associazione Sportiva Roma": "Roma", 
    "Terra e Vita": "Terra", 
    "Viva l'Italia!": "Italia", 
    "Che Dio ti benedica": "Dio",
    "Carne sintetica": "Coltura cellulare",
    "Storia dell'agricoltura italiana": "Agricoltura",
    "Grilli": "Gryllidae",
    "Macron": "Emmanuel Macron",
    "Malta Amateur Athletic Association": "Malta"
    }

In [4]:
def def_value(): 
    return 0

comments_entites_filtered2 = []
entities_freq_filtered2 = defaultdict(def_value)
input_file = "E:/Gianluca/Master Big Data Pisa/Progetto_Finale/Agricolo/Facebook/csv_docs_def/comments_entities.csv"
count = 0
with open(input_file, 'r', encoding='utf-8', newline='') as handle_r:
    reader = csv.DictReader(handle_r)
    i = 0
    for row in reader:
        row_entities = ast.literal_eval(row["entities"])
        comments_entites_filtered2.append([int(row["index"]), []])
        if len(row_entities) == 0:
            continue
        else:
            try:
                for entity in row_entities:
                    if entity["title"] in CORRECTIONS:
                        comments_entites_filtered2[i][1].append({"spot":entity["spot"], "start":entity["start"], "end":entity["end"], "title":CORRECTIONS[entity["title"]]})
                        entities_freq_filtered2[CORRECTIONS[entity["title"]]] = entities_freq_filtered2[CORRECTIONS[entity["title"]]] + 1
                        continue
                    if entity["title"] in ENTITIES_BIN:
                        continue
                    if entities_freq[entity["title"]] < 37:
                        if entities_freq[entity["title"]] >= 5 and entity["rho"] >= 0.3 and entity["link_probability"] >= 0.1:
                            comments_entites_filtered2[i][1].append(entity)
                            entities_freq_filtered2[entity["title"]] = entities_freq_filtered2[entity["title"]] + 1
                    else:
                        comments_entites_filtered2[i][1].append(entity)
                        entities_freq_filtered2[entity["title"]] = entities_freq_filtered2[entity["title"]] + 1
            except:
                count = count + 1
        i = i + 1

In [5]:
MODEL_FILE = r"E:/Gianluca/Master Big Data Pisa/Progetto_Finale/Agricolo 2/Group_6_2024_Project/Facebook/itwiki_20180420_100d.pkl/itwiki_20180420_100d.pkl"
wiki2vec = Wikipedia2Vec.load(MODEL_FILE)

c:\Users\giang\anaconda3\envs\env_master\Lib\site-packages\joblib\numpy_pickle.py:230: UserWarning: The memmapped array [[107  97]
 [124 112]
 [122 108]
 ...
 [  0   0]
 [  0   0]
 [  0   0]] loaded from the file E:/Gianluca/Master Big Data Pisa/Progetto_Finale/Agricolo 2/Group_6_2024_Project/Facebook/itwiki_20180420_100d.pkl/itwiki_20180420_100d.pkl is not byte aligned. This may cause segmentation faults if this memmapped array is used in some libraries like BLAS or PyTorch. To get rid of this warning, regenerate your pickle file with joblib >= 1.2.0. See https://github.com/joblib/joblib/issues/563 for more details
  warnings.warn(message)
c:\Users\giang\anaconda3\envs\env_master\Lib\site-packages\joblib\numpy_pickle.py:230: UserWarning: The memmapped array [[ 116589   53012]
 [ 134782   62437]
 [6811053  887006]
 ...
 [      0       0]
 [      0       0]
 [      0       0]] loaded from the file E:/Gianluca/Master Big Data Pisa/Progetto_Finale/Agricolo 2/Group_6_2024_Project/Facebook/

In [6]:
def get_entity_vector(e):
    try:
        emb = wiki2vec.get_entity_vector(e)
    except:
        raise Exception("Entity vector {} not found\n".format(e))
    return emb

In [7]:
entities_embeddings = []
embedding_counts = []
lost_entities = []
embeddings_dict = {}
embeddings_entities_dict = {}
for entity in entities_freq_filtered2:
    try:
        embedding = list(get_entity_vector(entity))
        embeddings_dict[entity] = embedding
        embeddings_entities_dict[tuple(embedding)] = entity
        entities_embeddings.append(embedding)
        embedding_counts.append(entities_freq_filtered2[entity])
    except:
        lost_entities.append(entity)

In [10]:
pca = PCA(n_components=3)
pca.fit(entities_embeddings)
embeddings_pca = pca.transform(entities_embeddings)

In [13]:
scaler = Normalizer()
X_minmax = scaler.fit_transform(entities_embeddings)

In [16]:
kmeans = KMeans(n_clusters=8, n_init=10, max_iter=100, random_state=42)
kmeans.fit(X_minmax, sample_weight=embedding_counts)

KMeans(max_iter=100, n_init=10, random_state=42)

In [17]:
centers = kmeans.cluster_centers_

def find_closest_vector(target_vector, vectors):
    # Compute the Euclidean distance from the target vector to all other vectors
    distances = np.linalg.norm(vectors - target_vector, axis=1)
    # Identify the index of the closest vector
    closest_index = np.argmin(distances)
    # Retrieve the closest vector
    closest_vector = vectors[closest_index]
    return closest_vector, closest_index

In [20]:
entity_centers = []
cluster_num_centers = {}
for center in centers:
    #closest_vector, closest_index = find_closest_vector(scaler.inverse_transform(center.reshape(1, -1)), np.array(entities_embeddings))
    closest_vector, closest_index = find_closest_vector(center, X_minmax)
    #closest_entity = embeddings_entities_dict[tuple(closest_vector)]
    closest_entity = embeddings_entities_dict[tuple(entities_embeddings[closest_index])]
    if closest_entity == "Ortaggio":
        entity_centers.append("Suolo")
        cluster_num_centers[kmeans.labels_[closest_index]] = "Suolo"
    else:
        entity_centers.append(closest_entity)
        cluster_num_centers[kmeans.labels_[closest_index]] = closest_entity

In [27]:
cluster_num_centers

{0: 'Roma',
 1: 'Agricoltura',
 2: 'Dio',
 3: 'Sindacato',
 4: 'Suolo',
 5: 'Società (sociologia)',
 6: 'Italia',
 7: 'Europa'}

In [23]:
embeddings_df = pd.DataFrame(embeddings_pca, columns=['x', 'y', 'z'])
embeddings_df["cluster"] = kmeans.labels_

In [24]:
entities_in_df = []
cluster_names_in_df = []
for embedding in entities_embeddings:
    entity = embeddings_entities_dict[tuple(embedding)]
    entities_in_df.append(entity)
for label in kmeans.labels_:
    cluster_names_in_df.append(cluster_num_centers[label])

In [25]:
embeddings_df["entity"] = entities_in_df
embeddings_df["cluster_name"] = cluster_names_in_df

In [34]:
# Menù a tendina per "Tipo dato"
dropdown_options1 = sorted(embeddings_df['cluster_name'].unique().tolist())

# Create a selection element for clusters
cluster_selection = alt.selection_single(
    fields=['cluster_name'],
    bind=alt.binding_select(options=embeddings_df['cluster_name'].unique(), name='Cluster: '),
    value = dropdown_options1[7]
)

# Create the scatter plot
scatter_plot = alt.Chart(embeddings_df).mark_circle().encode(
    x='x:Q',
    y='y:Q',
    color=alt.condition(
        cluster_selection,
        alt.Color('cluster_name:N', legend=None),
        alt.value('lightgrey')
    ),
    tooltip=['cluster_name', 'entity']
).add_selection(
    cluster_selection
).properties(
    title='Scatter Plot of Entity Embeddings with Cluster Selection',
    width=800,
    height=600
)

# Display the chart
scatter_plot.display()

c:\Users\giang\anaconda3\envs\env_master\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\giang\anaconda3\envs\env_master\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)